In [0]:
from tqdm import tqdm_notebook, tnrange as tqdm , tnrange
import pandas as pd
import numpy as np
from google.colab import drive

In [0]:
###Mount drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
###Load data from drive
impli_matrix=pd.read_csv("drive/Shared drives/Chula Drive/Datasets/AnimeRec-dataset/impli_matrix.csv")
impli_matrix=impli_matrix.set_index("user_id")
impli_matrix

,&quot;Bungaku Shoujo&quot; Movie,.hack//G.U. Trilogy,.hack//The Movie: Sekai no Mukou ni,009 Re:Cyborg,00:08,1000-nen Joou: Queen Millennia,1001 Nights,11-nin Iru!,12-gatsu no Uta,3-tsu no Kumo,663114,A.LI.CE,Aa! Megami-sama! Movie,Aachi wa Ssipak,Accel World: Infinite∞Burst,Acchi Kocchi,Ace wo Nerae! (1979),Adachi-ga Hara,Afro Samurai Movie,Afro Samurai: Resurrection,Ahiru no Otegara,Ai,Ai City,Aikatsu Stars! Movie,Aikatsu! Movie,Aikatsu! Music Award: Minna de Shou wo MoracchaimaShow!,Air Movie,Aitsu to Lullaby: Suiyobi no Cinderella,Ajin Part 1: Shoudou,Ajin Part 2: Shoutotsu,Ajin Part 3: Shougeki,Akage no Anne: Green Gables e no Michi,Aki no Kanade,Aki no Puzzle,Akihabara Dennou-gumi: 2011-nen no Natsuyasumi,Akira,Alexander Senki Movie,Ali Baba to 40-hiki no Touzoku,Alps no Shoujo Heidi (1979),Alps no Shoujo Heidi: Alm no Yama-hen,...,Watashi no Kao,Windaria,Wolf Daddy,Wonder (Movie),Wonderful Days,X/1999,Xabungle Graffiti,Xi Avant,Xiao Qian,Yamato yo Towa ni,Yami wo Mitsumeru Hane,Yatterman the Movie: Shin Yattermecha Osu Gou! Omocha no Kuni de Dai Ketsudan da Koron,Yawara! Sore Yuke Koshinuke Kids!!,Yes! Precure 5 GoGo! Movie: Okashi no Kuni no Happy Birthday,Yes! Precure 5 Movie: Kagami no Kuni no Miracle Daibouken!,Yodaka no Hoshi,Yoru no Okite,Youjuu Toshi,Youkai Watch Movie 1: Tanjou no Himitsu da Nyan!,Youkai Watch Movie 2: Enma Daiou to Itsutsu no Monogatari da Nyan!,Youkoso Lodoss-tou e!,Youkoso Uchuujin,Yousei Florence,Yowamushi Pedal Movie,Yowamushi Pedal: Re:RIDE,Yowamushi Pedal: Re:ROAD,Yu Bang Xiang Zheng,Yuki no Yo no Yume,Yume 10 Yoru,Yuureisen,Yuuyake Dandan,Yuu☆Yuu☆Hakusho (Movie),Yuu☆Yuu☆Hakusho: Meikai Shitou-hen - Honoo no Kizuna,Yu☆Gi☆Oh! (1999),Yu☆Gi☆Oh!: Hikari no Pyramid,Yu☆Gi☆Oh!: The Dark Side of Dimensions,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,eX-Driver the Movie,gdgd Fairies Movie: tte Iu Eiga wa Dou kana...?,xxxHOLiC Movie: Manatsu no Yoru no Yume
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9353,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9355,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
###Calculate pearson correlation between column
corr_matrix=impli_matrix.corr()
corr_matrix

,&quot;Bungaku Shoujo&quot; Movie,.hack//G.U. Trilogy,.hack//The Movie: Sekai no Mukou ni,009 Re:Cyborg,00:08,1000-nen Joou: Queen Millennia,1001 Nights,11-nin Iru!,12-gatsu no Uta,3-tsu no Kumo,663114,A.LI.CE,Aa! Megami-sama! Movie,Aachi wa Ssipak,Accel World: Infinite∞Burst,Acchi Kocchi,Ace wo Nerae! (1979),Adachi-ga Hara,Afro Samurai Movie,Afro Samurai: Resurrection,Ahiru no Otegara,Ai,Ai City,Aikatsu Stars! Movie,Aikatsu! Movie,Aikatsu! Music Award: Minna de Shou wo MoracchaimaShow!,Air Movie,Aitsu to Lullaby: Suiyobi no Cinderella,Ajin Part 1: Shoudou,Ajin Part 2: Shoutotsu,Ajin Part 3: Shougeki,Akage no Anne: Green Gables e no Michi,Aki no Kanade,Aki no Puzzle,Akihabara Dennou-gumi: 2011-nen no Natsuyasumi,Akira,Alexander Senki Movie,Ali Baba to 40-hiki no Touzoku,Alps no Shoujo Heidi (1979),Alps no Shoujo Heidi: Alm no Yama-hen,...,Watashi no Kao,Windaria,Wolf Daddy,Wonder (Movie),Wonderful Days,X/1999,Xabungle Graffiti,Xi Avant,Xiao Qian,Yamato yo Towa ni,Yami wo Mitsumeru Hane,Yatterman the Movie: Shin Yattermecha Osu Gou! Omocha no Kuni de Dai Ketsudan da Koron,Yawara! Sore Yuke Koshinuke Kids!!,Yes! Precure 5 GoGo! Movie: Okashi no Kuni no Happy Birthday,Yes! Precure 5 Movie: Kagami no Kuni no Miracle Daibouken!,Yodaka no Hoshi,Yoru no Okite,Youjuu Toshi,Youkai Watch Movie 1: Tanjou no Himitsu da Nyan!,Youkai Watch Movie 2: Enma Daiou to Itsutsu no Monogatari da Nyan!,Youkoso Lodoss-tou e!,Youkoso Uchuujin,Yousei Florence,Yowamushi Pedal Movie,Yowamushi Pedal: Re:RIDE,Yowamushi Pedal: Re:ROAD,Yu Bang Xiang Zheng,Yuki no Yo no Yume,Yume 10 Yoru,Yuureisen,Yuuyake Dandan,Yuu☆Yuu☆Hakusho (Movie),Yuu☆Yuu☆Hakusho: Meikai Shitou-hen - Honoo no Kizuna,Yu☆Gi☆Oh! (1999),Yu☆Gi☆Oh!: Hikari no Pyramid,Yu☆Gi☆Oh!: The Dark Side of Dimensions,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,eX-Driver the Movie,gdgd Fairies Movie: tte Iu Eiga wa Dou kana...?,xxxHOLiC Movie: Manatsu no Yoru no Yume
&quot;Bungaku Shoujo&quot; Movie,1.000000,0.103366,0.087181,0.134314,0.034685,-0.003199,0.035797,0.031016,-0.001847,-0.001847,0.064530,0.041115,0.118589,0.030317,-0.006130,-0.004889,-0.004131,0.074897,0.040182,0.053746,-0.002612,-0.002612,0.026816,-0.001847,0.021920,-0.003199,0.130066,-0.001847,0.026682,-0.003695,-0.003199,0.051658,0.104264,NaN,0.015768,0.074215,-0.002612,-0.001847,-0.003695,-0.001847,...,0.074897,0.108328,-0.003199,0.051658,0.087039,0.077428,-0.001847,0.115575,0.040181,0.098861,-0.001847,-0.001847,0.149825,-0.004526,-0.004889,-0.002612,-0.001847,0.043308,-0.003695,-0.001847,-0.006130,-0.001847,-0.003695,0.026096,0.017025,0.024130,-0.001847,-0.001847,NaN,-0.001847,NaN,0.037358,0.035495,0.025179,0.034879,-0.001847,0.105385,0.005182,-0.002612,0.133068
.hack//G.U. Trilogy,0.103366,1.000000,0.342153,0.057419,0.035920,-0.003104,0.052923,0.012594,-0.001792,-0.001792,0.031329,0.088122,0.143684,0.073807,0.017887,-0.004743,0.031329,-0.001792,0.050162,0.094383,-0.002534,-0.002534,0.027869,-0.001792,-0.005071,-0.003104,0.131309,-0.001792,0.002092,-0.003584,-0.003104,-0.002534,0.034424,NaN,0.039430,0.083164,0.053326,-0.001792,-0.003584,-0.001792,...,-0.001792,0.050350,-0.003104,-0.002534,0.067887,0.126367,-0.001792,0.019326,0.017887,0.031329,-0.001792,-0.001792,0.035920,0.027869,0.025126,-0.002534,-0.001792,0.010262,-0.003584,-0.001792,0.017887,-0.001792,-0.003584,0.009663,-0.005947,-0.004743,-0.001792,-0.001792,NaN,-0.001792,NaN,0.047683,0.069452,0.062372,0.070344,-0.001792,0.009359,0.082051,-0.002534,0.096623
.hack//The Movie: Sekai no Mukou ni,0.087181,0.342153,1.000000,0.143750,0.070506,-0.001666,-0.004911,0.032395,-0.000962,-0.000962,0.062637,-0.001666,0.056728,-0.005451,0.040507,-0.002545,-0.002151,-0.000962,0.046095,0.032896,-0.001360,-0.001360,-0.002356,-0.000962,-0.002721,-0.001666,0.049299,-0.000962,0.018006,-0.001923,-0.001666,-0.001360,0.048705,NaN,0.038507,0.032526,0.101057,-0.000962,-0.001923,-0.000962,...,-0.000962,0.084552,-0.001666,-0.001360,0.039805,0.048149,-0.000962,0.042785,-0.003191,-0.002151,-0.000962,-0.000

In [0]:
###get rid of self correlation
np.fill_diagonal(corr_matrix.values,np.nan)

### sort for top five correlation ( argsort sort min to max so we put minus on the dataset and return index)
order_top5=np.argsort(-corr_matrix.values, axis=1)[:,:5]

### sort for worst correlation
order_worst=np.argsort(corr_matrix.values, axis=1)[:,:1]

### create top5 data frame
result_top5 = pd.DataFrame(
    corr_matrix.columns[order_top5], 
    columns=['1st', '2nd', '3rd', '4th', '5th'],
    index=corr_matrix.index
)

### create worst data frame
result_bottom = pd.DataFrame(
    corr_matrix.columns[order_worst], 
    columns=['worst'],
    index=corr_matrix.index
)

### join both dataframe
result=result_top5.join(result_bottom)

### look up for the corr score of the topfive and worst and put up with the result dataframe
for x in result.columns:
    result[x+"_Val"] = corr_matrix.lookup(corr_matrix.index, result[x])
result

In [0]:
#### Weight sum and find top 3
def user_predicted(user_row):
  user_data=impli_matrix.iloc[[user_row],:].values.flatten()
  voted_item=np.where(user_data==1)[0]
  unvoted_item=np.where(user_data==0)[0]
  dict_score_columns={}
  for i in unvoted_item:
    sum_corr_score=0
    sum_weight=0
    abs_corr_score=0
    i_name=corr_matrix.columns[i]
    for j in voted_item:
     corr_score=corr_matrix.iloc[[i],[j]].values.flatten()[0]
     if (corr_score==0):
       break
     sum_corr_score+=corr_score
     abs_corr_score+=np.abs(corr_score)
     sum_weight+=sum_corr_score/abs_corr_score
    dict_score_columns[sum_weight]=i_name
  top_keys=sorted(dict_score_columns.keys())
  result=[]
  for i in top_keys[:3]:
   result.append(dict_score_columns[i])
  return result

In [0]:
user_predicted(2)